In [1]:
import matplotlib.pyplot as plt
import numpy as np
import math
import pandas as pd
import statsmodels.api as sm
from scipy.special import gamma, loggamma, factorial
import scipy.stats
from scipy.interpolate import interp1d
from matplotlib.ticker import AutoMinorLocator 
from matplotlib import rc, font_manager
from matplotlib.lines import Line2D
from matplotlib import colors as mcolors
from mycolorpy import colorlist as mcp
from matplotlib import legend_handler
from mpl_toolkits.axes_grid1 import make_axes_locatable, axes_size
import collections
import matplotlib
import shapely.geometry
import shapely.ops 
from geopy.geocoders import Nominatim
import geopandas as gpd
import mapclassify
import jenkspy
import contextily as cx
import os
import rioxarray as rx
import pyreadr
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn import metrics
import plots
import seaborn as sns
from prophet import Prophet
import datetime
import warnings
warnings.filterwarnings("ignore")

In [2]:
wd = os.path.dirname(os.getcwd())

In [177]:
country = "CHL"

if country == "ARG":
    country_full = "argentina"
elif country == "CHL":
    country_full = "chile"
elif country == "COL":
    country_full = "colombia"
elif country == "MEX":
    country_full = "mexico"

In [178]:
date_list = []
years = ['2020', '2021', '2022']
months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
for year in years:
    i = 0
    if year == '2020':
        i = 3
    if year == '2022':
        max_month = 5
    else:
        max_month = 12
    while i < max_month:
        date_list.append(year + '-' + str(months[i]) + '-01')
        i += 1


In [179]:
date_list

['2020-04-01',
 '2020-05-01',
 '2020-06-01',
 '2020-07-01',
 '2020-08-01',
 '2020-09-01',
 '2020-10-01',
 '2020-11-01',
 '2020-12-01',
 '2021-01-01',
 '2021-02-01',
 '2021-03-01',
 '2021-04-01',
 '2021-05-01',
 '2021-06-01',
 '2021-07-01',
 '2021-08-01',
 '2021-09-01',
 '2021-10-01',
 '2021-11-01',
 '2021-12-01',
 '2022-01-01',
 '2022-02-01',
 '2022-03-01',
 '2022-04-01',
 '2022-05-01']

In [180]:
df_stringency = pd.read_csv("/Volumes/RECAST/data/covid_stringency/owid-covid-data.csv")
df_stringency = df_stringency[df_stringency['location'] == str(country_full).capitalize()].reset_index(drop=True)
df_stringency = df_stringency.filter(['location', 'date', 'stringency_index'], axis=1)
df_stringency = df_stringency.fillna(0)
df_stringency['year'] = [int(df_stringency['date'][i][0:4]) for i in range(len(df_stringency))]
df_stringency['month'] = [int(df_stringency['date'][i][5:7]) for i in range(len(df_stringency))]

stringencies = []
dss = []
start_date = '2020-03-01'
end_date = '2023-02-01'
date_range = pd.date_range(start=start_date, end=end_date, freq='MS')
df_stringency_month = pd.DataFrame({'ds': date_range})
for i in range(len(df_stringency_month)):
    year = df_stringency_month.loc[i, 'ds'].year
    month = df_stringency_month.loc[i, 'ds'].month
    df_stringency_filter = df_stringency[df_stringency['year'] == year].reset_index(drop=True)
    df_stringency_filter = df_stringency_filter[df_stringency_filter['month'] == month].reset_index(drop=True)
    stringencies.append(np.mean(df_stringency_filter['stringency_index']))
    

In [181]:
for i in gdf_admin_mov.columns:
    print(i)

GID_2
GID_0
COUNTRY
GID_1
NAME_1
NL_NAME_1
NAME_2
VARNAME_2
NL_NAME_2
TYPE_2
ENGTYPE_2
CC_2
HASC_2
eFUA_ID
UC_num
UC_IDs
eFUA_name
Commuting
Cntry_ISO
Cntry_name
FUA_area
UC_area
FUA_p_2015
UC_p_2015
Com_p_2015
pop2020_FUA
den2020_FUA
rdi
pop2020
den2020
n_crisis_04_2020
n_baseline_04_2020
n_perchange_04_2020
n_crisis_06_2020
n_baseline_06_2020
n_perchange_06_2020
n_crisis_07_2020
n_baseline_07_2020
n_perchange_07_2020
n_crisis_08_2020
n_baseline_08_2020
n_perchange_08_2020
n_crisis_09_2020
n_baseline_09_2020
n_perchange_09_2020
n_crisis_10_2020
n_baseline_10_2020
n_perchange_10_2020
n_crisis_11_2020
n_baseline_11_2020
n_perchange_11_2020
n_crisis_12_2020
n_baseline_12_2020
n_perchange_12_2020
n_crisis_01_2021
n_baseline_01_2021
n_perchange_01_2021
n_crisis_02_2021
n_baseline_02_2021
n_perchange_02_2021
n_crisis_03_2021
n_baseline_03_2021
n_perchange_03_2021
n_crisis_04_2021
n_baseline_04_2021
n_perchange_04_2021
n_crisis_05_2021
n_baseline_05_2021
n_perchange_05_2021
n_crisis_06_2021


In [176]:
gdf_admin_mov = gpd.read_file(wd + '/data/outputs/' + country + '/gdf_' + country + '_admin_rdi_mov_evo.gpkg')

months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
years = ['2020', '2021', '2022']

for year in years:
    for month in months:  
        try:
            gdf_admin_mov = gdf_admin_mov[gdf_admin_mov['z_score_perchange_' + str(month) + '_' + str(year)] < 4].reset_index(drop=True)
        except:
            continue

    
df_evo_admins = {}
for i in range(len(gdf_admin_mov)):
    evo_admin = []
    for year in years:
        for month in months:
            try:
                evo_admin.append(gdf_admin_mov.loc[i, 'n_perchange_' + month + '_' + year]) 
            except:
                continue
    df_evo_admins[str(i)] = pd.DataFrame({'ds': date_list, 'y': evo_admin})
    
    months_evo = []
    for j in range(len(df_evo_admins[str(i)])):
        months_evo.append(datetime.datetime.strptime(df_evo_admins[str(i)]['ds'][j], '%Y-%m-%d').month)
    df_evo_admins[str(i)]['month'] = months_evo
    df_evo_admins[str(i)]['stringency'] = stringencies[0:len(df_evo_admins[str(i)])]
    

In [ ]:
fcst_admins = {}
for i in range(len(df_evo_admins)):
    m = Prophet(seasonality_mode='additive', yearly_seasonality=True)
#     m.add_regressor('month', standardize=False)
#     m.add_regressor('stringency', standardize=True)
    m.fit(df_evo_admins[str(i)])
    future = m.make_future_dataframe(periods=10, freq='MS')
#     future['month'] = future['ds'].dt.month
#     future['stringency'] = stringencies
    fcst = m.predict(future)
    fcst_admins[str(i)] = fcst

In [ ]:
fig,ax = plt.subplots(figsize=(8,7))

ax.spines['bottom'].set_linewidth(0)
ax.spines['top'].set_linewidth(0)
ax.spines['left'].set_linewidth(0)
ax.spines['right'].set_linewidth(0)

ax.tick_params(axis = 'both', which = 'both', width = 0, length = 0, color = 'k', labelsize=20, pad=9)

for i in range(len(fcst_admins)):
    fcst = fcst_admins[str(i)]
    ax.plot(np.arange(0,26), fcst['yhat'][:26], color='midnightblue', alpha=0.05)
    ax.plot(np.arange(25, len(fcst['ds'])), fcst['yhat'][25:], color='navy', alpha=0.1)
    
for k in range(len(fcst)):
#     try:
    rgba = matplotlib.cm.copper((stringencies[k]/2+50-min(stringencies))/max(stringencies))
#     except:
#         rgba = matplotlib.cm.copper(1-(stringencies[k-1]-min(stringencies))/max(stringencies))
    x = [k-.50, k+.50]
    ax.fill_between(x, 100*np.ones(len(x)), -100*np.ones(len(x)), color=rgba, alpha=1, edgecolor='None', linewidth=0, zorder=0)

  